In [1]:
# external import
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib.request
import time
from PIL import ImageFile
import re
from newspaper import Article
import newspaper
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np

In [2]:
def simple_get(url):
    """
    Transforms URL into a Soup object
    :param url: URL
    :rtype: BeautifulSoup Object
    """
    if valid_url(url):
        if url_is_alive(url):
            page = urlopen(url)
            soup = BeautifulSoup(page,"lxml")  #html.parser, html5lib, lxml
            return soup
        else:
            return "Bad URL"
    else:
        return "Bad URL"
    
def get_title(soup):
    """
    Extracts the title from the website
    :param soup: BeautifulSoup Object
    :rtype: String
    """
    return soup.title

def get_pictures(soup):
    """
    Takes first 4 valid image URL's from website and chooses the best picture by IMG size.
    :param soup: BeautifulSoup Object
    :rtype: Image URL of the selected IMG
    """
    image_tags = soup.findAll('img')
    counter = 0
    urls = []
    sizes = []
    # print out image urls
    for image_tag in image_tags:
            if valid_url(image_tag.get('src')) and counter < 4:
                counter += 1
                urls.append(image_tag.get('src'))
                sizes.append(getsizes(image_tag.get('src'))[0])
    return urls[sizes.index(max(sizes))]
        
def url_is_alive(url):
    """
    Checks that a given URL is reachable.
    :param url: A URL
    :rtype: bool
    """
    request = urllib.request.Request(url)
    request.get_method = lambda: 'HEAD'

    try:
        urllib.request.urlopen(request)
        return True
    except urllib.request.HTTPError:
        return False

def valid_url(url):
    """
    Checks if a URL is malformed
    :param url: A URL
    :rtype: bool
    """
    regex = re.compile(
            r'^(?:http|ftp)s?://' # http:// or https://
            r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
            r'localhost|' #localhost...
            r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
            r'(?::\d+)?' # optional port
            r'(?:/?|[/?]\S+)$', re.IGNORECASE)

    return re.match(regex, url) is not None   # True

def getsizes(url):
    """
    Takes a image URL and returns its size 
    :param url: A URL
    :rtype: Int
    """
    file = urlopen(url)
    size = file.headers.get("content-length")
    if size: size = int(size)
    p = ImageFile.Parser()
    while 1:
        data = file.read(1024)
        if not data:
            break
        p.feed(data)
        if p.image:
            return size, p.image.size
            break
    file.close()
    return size, None

def get_article_links(url):
    """
    Takes a URL with articles and returns a list of URLs of all the contained articles.
    :param url: A URL
    :rtype: list
    """
    links = newspaper.build(url, memoize_articles=False, language = 'zh')
    return [article.url for article in links.articles]

def recommend_article(article_feed_url, tag):
    """
    Takes a URL with articles as well as a tag and returns the recommended article title 
    (for now) from this URL based on the tag. Currently selects the article with highest
    tag occurrences in its main text. 
    :param article_feed_url: A URL
    :param tag: String
    :rtype: String
    """
    ##### alternative method with beautiful soup (not guaranteed to work yet) ######
    # for article in soup.findAll('a', attrs = {'href': re.compile('.*\.html$')}):
        # ...
        
    
    # to handle authentication information
    # create a password manager
#     password_mgr = urllib.request.HTTPPasswordMgrWithDefaultRealm()

#     # Add the username and password.
#     # If we knew the realm, we could use it instead of None.
#     password_mgr.add_password(None, article_feed_url, 'eddiewu0429@gmail.com', 'wsywsh1019')

#     handler = urllib.request.HTTPBasicAuthHandler(password_mgr)

#     # create "opener" (OpenerDirector instance)
#     opener = urllib.request.build_opener(handler)

#     # use the opener to fetch a URL
#     opener.open(article_feed_url)

#     # Install the opener.
#     # Now all calls to urllib.request.urlopen use our opener.
#     urllib.request.install_opener(opener)
    
    soup = simple_get(article_feed_url)
    list_of_article_titles = []
    all_text = []
    key_words = []
    
    try:
        for article in soup.findAll('a', href=True):
            if article.text and article['href'] and len(article.text.replace(' ', '')) >= 15:
                cur_article = Article(article_feed_url + article['href'][1:], language = 'zh')
                cur_article.download()
                cur_article.parse()
                cur_article.nlp()
                list_of_article_titles.append(cur_article.title)
                all_text.append(cur_article.text.lower())
                key_words.append(cur_article.keywords)

        print("there are in total of ", len(list_of_article_titles), ' articles collected')
        print("These are the titles of found articles: ", list_of_article_titles)

    except:
        print("download limit exceeded... but the result so far is returned...")
        print("there are in total of ", len(list_of_article_titles), ' articles collected')

    
    if not all_text:
        return None
    
    # create vector representation of our articles
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(all_text)
    print(X.shape)

    # binary target variable (whether or not the tag is in the keywords)
    y = []
    for keywords in key_words:
        check = False
        for keyword in keywords:
            if tag in keyword:
                check = True
                break
        if check:
            y.append(1)
        else:
            y.append(0)
    print(y)

    # build model to find article with highest probability
    clf = LogisticRegression().fit(X, y)
    article_probs = clf.predict_proba(X)[:, 1]
    print(article_probs)
    return list_of_article_titles[np.argmax(article_probs)]
    
    '''
    # below works well for english sites
    
    if valid_url(article_feed_url) and url_is_alive(article_feed_url):
        list_of_article_titles = []
        tag_frequency = []
        article_urls = get_article_links(article_feed_url) 
        print(article_urls)
        all_text = []
        try:
            for article_url in article_urls: 
                if valid_url(article_url) == False or url_is_alive(article_url) == False: 
                    continue
                cur_article = Article(article_url, language = 'zh')
                cur_article.download()
                cur_article.parse()
                list_of_article_titles.append(cur_article.title)
                tag_frequency.append(cur_article.text.lower().count(tag))
                all_text.append(cur_article.text.lower())
            print("there are in total of {0} articles collected".format(len(list_of_article_titles)))
            print("the tag occurs {0} times in the selected article".format(tag_frequency.index(max(tag_frequency))))
            print(all_text[tag_frequency.index(max(tag_frequency))])
            return list_of_article_titles[tag_frequency.index(max(tag_frequency))]
        except:
            print("download limit exceeded... but the result so far is returned...")
            print("there are in total of ", len(list_of_article_titles), ' articles collected')
            print("the tag occurs ", tag_frequency.index(max(tag_frequency)), " times in the selected article")
            print(all_text[tag_frequency.index(max(tag_frequency))])
            return list_of_article_titles[tag_frequency.index(max(tag_frequency))]
    else:
        return 'Bad URL'
    '''

In [3]:
# Test 1 to get best picture and article title
soup = simple_get("http://tieba.baidu.com/p/5858562576")
print("Best Picture ==>",get_pictures(soup))
print("Article Title ==>",get_title(soup))

Best Picture ==> http://imgsrc.baidu.com/forum/pic/item/d158ccbf6c81800a0cd02c07b63533fa828b4785.jpg
Article Title ==> <title>为热爱而战 网易电竞NeXT全面启动【竞技游戏吧】_百度贴吧</title>


In [4]:
# Test 2 to get best picture and article title
soup = simple_get("http://www.nba.com/")
print("Best Picture ==>",get_pictures(soup))
print("Article Title ==>",get_title(soup))

Best Picture ==> https://www.nba.com/assets/logos/turner-logo.svg
Article Title ==> <title>The official site of the NBA | NBA.com</title>


In [7]:
# Test recommend_article function
start = time.time()
print("Recommended Article Title ==>", recommend_article('https://www.cbnweek.com/','科技')) # https://www.cbnweek.com/
'{0} seconds taken to recommend this article from given site with given tag'.format(time.time()-start)

there are in total of  21  articles collected
These are the titles of found articles:  ['茅台连续大跌，是熊市的开始还是结束？', 'ofo日本将于10月底停止服务；美团调整架构， 组建用户平台和两大事业群', '改革开放40周年，这些“洋品牌”都是怎样献礼的', 'ZARA的 “平衡术”', 'ZARA的平衡术', 'Waymo开始收费，无人驾驶进入商业化时代？', 'IBM 340亿美元收购Red Hat，蓝色巨人转型仍前途未卜', '一向高冷的ZARA开始变得积极，发生了什么？', 'ZARA的 “平衡术”', 'ZARA的 “平衡术”', '麦肯锡首次发布设计商业价值调查，设计对公司到底有什么商业价值？', '消费者担心喝奶茶会发胖，喜茶用甜菊糖代替了冰糖糖浆', '要一眼识别出哪些是城市里品质更高的居住空间，数据能够做什么？', '全球股市经历黑色交易日；拼多多与天猫大促令商户再陷“二选一”难题', '长租公寓再暴雷 寓见公寓倒闭', '中国新式茶饮行业 品质严选之路', '第一财经周刊', 'AR照进零售店', '你的消费能力配得上“中产”二字吗？', '第一财经精英座驾评选', '第一财经周刊']
(21, 1350)
[0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
[0.01631014 0.97326403 0.0258246  0.09932186 0.10348676 0.93697266
 0.95267138 0.02034083 0.09932186 0.09932186 0.02119974 0.01944067
 0.01090251 0.9808992  0.93972657 0.16292663 0.16292663 0.14760219
 0.16292663 0.16292663 0.21236645]
Recommended Article Title ==> 全球股市经历黑色交易日；拼多多与天猫大促令商户再陷“二选一”难题


'37.87026596069336 seconds taken to recommend this article from given site with given tag'

In [56]:
import requests
payload = {'username': 'eddiewu0429@gmail.com', 'password': 'wsywsh1019'}
url = 'https://www.zhihu.com/signup?next=%2F'
r = requests.post(url, data=payload)
get_article_links(r.url)

['https://www.zhihu.com/question/20959801/answer/36695081',
 'http://www.cac.gov.cn/2000-09/30/c_126193701.htm',
 'http://www.cac.gov.cn/2016-06/25/c_1119109085.htm']

In [67]:
auth_handler = urllib.request.HTTPBasicAuthHandler()
auth_handler.add_password(None,'https://www.zhihu.com/signup?next=%2F','eddiewu0429@gmail.com','wsywsh1019')
opener = urllib.request.build_opener(auth_handler)
urllib.request.install_opener(opener)
req = urllib.request.Request(url='https://www.zhihu.com/')
r = urllib.request.urlopen(req)
get_article_links(r.url)

['https://www.zhihu.com/question/20959801/answer/36695081',
 'http://www.cac.gov.cn/2000-09/30/c_126193701.htm',
 'http://www.cac.gov.cn/2016-06/25/c_1119109085.htm']

In [63]:
import requests

url = 'https://www.zhihu.com/signup?next=%2F'
login_data = dict(username='eddiewu0429@gmail.com', password='wsywsh1019')
session = requests.session()

r = session.post(url, data=login_data)
r2 = session.get('https://www.zhihu.com/')
r2.url, r2.content

('https://www.zhihu.com/',
 b'<html>\r\n<head><title>400 Bad Request</title></head>\r\n<body bgcolor="white">\r\n<center><h1>400 Bad Request</h1></center>\r\n<hr><center>openresty</center>\r\n</body>\r\n</html>\r\n')

In [11]:
url = 'https://www.cbnweek.com/articles/normal/22603'

a = Article(url, language='zh') # Chinese

a.download()
a.parse()
a.text

'以前，十万人级别的基因组测序可能需要4到6年的时间才能完成，而现在，6台华大智造T7，一年时间即可完成。\n\n10月25日，在第十三届国际基因组学大会（International Conference on Genomics）上，作为主办方，华大集团子公司华大智造在开幕式上发布了一款自主研发的高通量基因测序仪——“MGISEQ-T7”。\n\n这台基因测序仪一天就可以完成60例个人全基因组测序，日产出数据高达6Tb（约6000G），是目前全球日生产能力最强的基因测序仪，被称为“超级生命计算机”。以前，十万人级别的基因组测序可能需要4到6年的时间才能完成，而现在，6台华大智造T7，一年时间即可完成。\n\n华大基因是一个专门从事生命科学的科技前沿机构，涉及人类、医学、农业、畜牧、濒危动物保护等分子遗传层面的科技研究。此次高通量基因测序仪的发布，同时也意味着华大集团成为了全球唯一一个有能力构建一条从硬件制造到基因测序服务、再到大数据研究的完整基因测序产业链的机构。\n\n然而，就在华大发布这款重磅产品前一天，10月24日，科技部首次公开了包括其在内的6家公司或机构，因违反人类遗传资源管理规定所受到的处罚。\n\n其中，华大基因和复旦大学附属华山医院未经许可与英国牛津大学开展“中国女性单相抑郁症的大样本病例对照研究”的中国人类遗传资源国际合作研究。而《人类遗传资源管理暂行办法》第四条规定，国家对重要遗传家系和特定地区遗传资源实行申报登记制度；未经许可，任何单位和个人不得擅自采集、收集、买卖、出口、出境或以其他形式对外提供。\n\n华大基因回应称，公司在2015年收到该行政处罚后，第一时间整改，科技部也已批准华大基因恢复开展人类遗传资源国际合作工作。\n\n华大基因2017年年报显示，该公司在中国内地实现营业收入15.96亿元，占比76.17%，非内地业务占比23.83%。其中，欧洲、非洲及中东的营收占比最高，占公司总体营收的9.46%。'

In [8]:
import math
int(math.log(34, 10))

1

In [11]:
num = 34
n = 1
num // 10 ** (int(math.log(num, 10)) - n + 1)

3

In [12]:
int(str(10) + str(20))

1020

In [61]:
a = ['字节跳动完成Pre-IPO融资，估值超百度京东', '华大基因发布新基因测序仪，一天可完成60例个人全基因组测序', '什么？你在疯抢一间出版社的周边？', '麦肯锡首次发布设计商业价值调查，设计对公司到底有什么商业价值？', 'ZARA的 “平衡术”', 'ZARA的平衡术', 'Gucci连续7个季度收入增长逾35%，却是两年来最慢增长', '国泰航空940万乘客资料外泄；Bilibili与腾讯宣布达成ACG内容合作', '为防止马蜂窝数据造假事件发生，投资人的尽职调查该怎么做？', 'ZARA的 “平衡术”', 'ZARA的 “平衡术”', '麦肯锡首次发布设计商业价值调查，设计对公司到底有什么商业价值？', '23本编辑部荐书陪你过长假', '100 个有意思的人（13）', '优衣库创始人柳井正：我们要改变服装、改变常识', '马斯克辞去特斯拉董事长职位；Facebook有5000万个账号存在安全漏洞', '第一财经周刊', 'AR照进零售店', '你的消费能力配得上“中产”二字吗？', '第一财经精英座驾评选', '新一线武汉特刊', '第一财经周刊']
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(a)
print(vectorizer.get_feature_names(), len(vectorizer.get_feature_names()))
print(X.toarray(), X.shape)  

['100', '13', '23本编辑部荐书陪你过长假', 'ar照进零售店', 'bilibili与腾讯宣布达成acg内容合作', 'facebook有5000万个账号存在安全漏洞', 'gucci连续7个季度收入增长逾35', 'ipo融资', 'zara的', 'zara的平衡术', '一天可完成60例个人全基因组测序', '个有意思的人', '中产', '为防止马蜂窝数据造假事件发生', '二字吗', '什么', '优衣库创始人柳井正', '估值超百度京东', '你在疯抢一间出版社的周边', '你的消费能力配得上', '华大基因发布新基因测序仪', '却是两年来最慢增长', '国泰航空940万乘客资料外泄', '字节跳动完成pre', '平衡术', '我们要改变服装', '投资人的尽职调查该怎么做', '改变常识', '新一线武汉特刊', '第一财经周刊', '第一财经精英座驾评选', '设计对公司到底有什么商业价值', '马斯克辞去特斯拉董事长职位', '麦肯锡首次发布设计商业价值调查'] 34
[[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 